### New Edit

__Additions__
- A logic to continuously check the equity/balance  
- All trades are automatically closed when the target is reached (Target = Initial balance * 200GHC)
- An extra logic to manage losing trades and wining trade having opposite trade signal activated.

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MAGIC_NUMBER = 10003
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 200  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === CHECK PROFIT AND CLOSE ALL TRADES ===
def check_profit_and_exit():
    global INITIAL_BALANCE
    account_info = mt5.account_info()
    if account_info is None:
        logging.error("❌ Could not fetch account info.")
        return

    net_profit = account_info.balance - INITIAL_BALANCE
    logging.info(f"💹 Current Net Profit: ${net_profit:.2f}")

    if net_profit >= PROFIT_TARGET:
        logging.info("🎯 Profit target reached. Closing all positions...")
        positions = mt5.positions_get()
        if positions:
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": pos.symbol,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "position": pos.ticket,
                    "price": mt5.symbol_info_tick(pos.symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(pos.symbol).ask,
                    "deviation": 20,
                    "magic": MAGIC_NUMBER,
                    "comment": "Close on profit target",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    logging.info(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    logging.warning(f"⚠️ Failed to close position {pos.ticket}: {result.retcode}")
        logging.info("🛑 Stopping bot after reaching profit target.")
        mt5.shutdown()
        exit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(df, period=14):
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = df['true_range'].rolling(period).mean()
    return df

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    atr = df.iloc[-1]['ATR']
    support = df['low'].rolling(10).min().iloc[-1]
    resistance = df['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return round(stop_loss, 2), round(take_profit, 2)

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if df.iloc[-1]['high'] > df.iloc[-2]['high']:
        signal = "BUY"
        entry_price = df.iloc[-1]['close']
    elif df.iloc[-1]['low'] < df.iloc[-2]['low']:
        signal = "SELL"
        entry_price = df.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def equity_check():
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return
    
    inital_balance = account_info.balance
    target_equity = inital_balance + 200
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is not None:
            equity = account_info.equity

            if equity >= target_equity:
                print("🚀 Equity target reached! Closing all open positions...")
                positions = mt5.positions_get()
                if positions:
                    for pos in positions:
                        symbol = pos.symbol
                        volume = pos.volume
                        ticket = pos.ticket
                        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                        price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                        close_request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": ticket,
                            "symbol": symbol,
                            "volume": volume,
                            "type": order_type,
                            "price": price,
                            "deviation": 10,
                            "magic": 123456,
                            "comment": "Auto close on equity profit"
                        }

                        result = mt5.order_send(close_request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"✔️ Closed position {ticket}")
                        else:
                            print(f"❌ Failed to close position {ticket}, retcode: {result.retcode}")
        else:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")

        time.sleep(60)

# === RUN THE BOT ===
def run_bot():
    connect_mt5()
    initial_equity = mt5.account_info().equity  # Get initial equity of the account

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check, daemon=True).start()
    
    while True:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Check if equity has reached the target profit
        current_equity = mt5.account_info().equity
        if current_equity >= initial_equity + PROFIT_TARGET:
            print(f"✅ Equity reached ${PROFIT_TARGET} profit. Closing all positions...")
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": pos.ticket,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "price": mt5.symbol_info_tick(SYMBOL).ask if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).bid,
                    "deviation": 10,
                    "magic": 123456,
                    "comment": "Closing positions after reaching profit target",
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    print(f"❌ Failed to close position {pos.ticket}: {result.retcode}")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")

        time.sleep(300)


# Start the trading bot
run_bot()


✅ MT5 initialized and logged in successfully.
✅ Initial Balance: 1000.0
🎯 Monitoring for equity to reach $1200.00...
📢 Trade Signal: BUY | Entry: 3288.521 | SL: 3283.12 | TP: 3299.32
✅ BUY Order Placed: 3288.521
❌ Failed to update SL/TP: 10016
✅ BUY SL updated for position 1881449899 → 3.56
⏳ No trade opportunity found.
✅ BUY SL updated for position 1881449899 → 8.8
⏳ No trade opportunity found.
✅ BUY SL updated for position 1881449899 → 12.84
⏳ No trade opportunity found.
✅ BUY SL updated for position 1881449899 → 16.97
📢 Trade Signal: SELL | Entry: 3278.366 | SL: 3288.35 | TP: 3258.39
✅ SELL Order Placed: 3278.366
✅ SL/TP Updated: SL=3288.35, TP=3258.39
✅ BUY SL updated for position 1881449899 → 20.87
✅ SELL SL updated for position 1881509064 → 3288.12
⚠️ Trade skipped: Already in a SELL position.
⏳ Waiting for new opposite signal...
✅ BUY SL updated for position 1881449899 → 27.35
⚠️ Trade skipped: Already in a BUY position.
⏳ Waiting for new opposite signal...
✅ BUY SL updated for 

### Correction

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MAGIC_NUMBER = 10003
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 200  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login
stop_trading = False  # Shared flag


# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(df, period=14):
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = df['true_range'].rolling(period).mean()
    return df

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    atr = df.iloc[-1]['ATR']
    support = df['low'].rolling(10).min().iloc[-1]
    resistance = df['high'].rolling(10).max().iloc[-1]
    sl_pips = 680
    sl_distance = sl_pips * LOT_SIZE
    
    if pd.isna(atr) or pd.isna(support) or pd.isna(resistance):
        return None, None

    if atr < 5:
        if signal == "BUY":
            stop_loss = max(support, entry_price - (atr * 1.5))
            take_profit = entry_price + ((entry_price - stop_loss) * 2)
        else:
            stop_loss = min(resistance, entry_price + (atr * 1.5))
            take_profit = entry_price - ((stop_loss - entry_price) * 2)
    else:
        if signal == "BUY":
            stop_loss = entry_price - sl_distance
            take_profit = entry_price + (sl_distance * 2)
        else:
            stop_loss = entry_price + sl_distance
            take_profit = entry_price - (sl_distance * 2)

    return round(stop_loss, 2), round(take_profit, 2)

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if df.iloc[-1]['high'] > df.iloc[-2]['high']:
        signal = "BUY"
        entry_price = df.iloc[-1]['close']
    elif df.iloc[-1]['low'] < df.iloc[-2]['low']:
        signal = "SELL"
        entry_price = df.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def equity_check_and_close_trades():
    global stop_trading
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return

    INITIAL_BALANCE = account_info.balance
    target_equity = INITIAL_BALANCE + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while not stop_trading:
        account_info = mt5.account_info()
        if account_info is None:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")
            time.sleep(60)
            continue

        equity = account_info.equity
        print(f"🔍 Current Equity: ${equity:.2f} | Target: ${target_equity:.2f}")

        if equity >= target_equity:
            print("🚀 Equity target reached! Closing all open positions...")
            positions = mt5.positions_get()
            if positions is None or len(positions) == 0:
                print("ℹ️ No open positions to close.")
            else:
                for pos in positions:
                    symbol = pos.symbol
                    volume = pos.volume
                    ticket = pos.ticket
                    order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                    price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                    close_request = {
                        "action": mt5.TRADE_ACTION_DEAL,
                        "position": ticket,
                        "symbol": symbol,
                        "volume": volume,
                        "type": order_type,
                        "price": price,
                        "deviation": 10,
                        "magic": 123456,
                        "comment": "Auto close on equity profit"
                    }

                    result = mt5.order_send(close_request)
                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"✅ Position {ticket} closed successfully.")
                    else:
                        print(f"❌ Failed to close position {ticket}: {result.retcode}")

            stop_trading = True  # Set the stop flag
            break

        time.sleep(60)

# === RUN THE BOT ===
def run_bot():
    global stop_trading
    connect_mt5()

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check_and_close_trades, daemon=True).start()
    
    while not stop_trading:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")
    
        time.sleep(300)
    
    print("🛑 Trading stopped due to equity target being reached.")

# Start the trading bot
run_bot()


✅ MT5 initialized and logged in successfully.
✅ Initial Balance: 883.35
🎯 Monitoring for equity to reach $1083.35...
🔍 Current Equity: $883.35 | Target: $1083.35
📢 Trade Signal: BUY | Entry: 3219.869 | SL: 3213.07 | TP: 3233.47
❌ Trade Failed: 10018
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
📢 Trade Signal: BUY | Entry: 3238.117 | SL: 3231.32 | TP: 3251.72
❌ Trade Failed: 10031
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
📢 Trade Signal: BUY | Entry: 3238.117 | SL: 3231.32 | TP: 3251.72
❌ Trade Failed: 10018
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target: $1083.35
🔍 Current Equity: $883.35 | Target

### Adjusted for 4.15 ATR(Low Volatility) and Equity close at 100,

- Allows multiple trades in same directions.
- Closes trades when opposite signal is found.

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MAGIC_NUMBER = 10003
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 100  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === CHECK PROFIT AND CLOSE ALL TRADES ===
def check_profit_and_exit():
    global INITIAL_BALANCE
    account_info = mt5.account_info()
    if account_info is None:
        logging.error("❌ Could not fetch account info.")
        return

    net_profit = account_info.balance - INITIAL_BALANCE
    logging.info(f"💹 Current Net Profit: ${net_profit:.2f}")

    if net_profit >= PROFIT_TARGET:
        logging.info("🎯 Profit target reached. Closing all positions...")
        positions = mt5.positions_get()
        if positions:
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": pos.symbol,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "position": pos.ticket,
                    "price": mt5.symbol_info_tick(pos.symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(pos.symbol).ask,
                    "deviation": 20,
                    "magic": MAGIC_NUMBER,
                    "comment": "Close on profit target",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    logging.info(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    logging.warning(f"⚠️ Failed to close position {pos.ticket}: {result.retcode}")
        logging.info("🛑 Stopping bot after reaching profit target.")
        mt5.shutdown()
        exit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(df, period=14):
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = df['true_range'].rolling(period).mean()
    return df

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    atr = df.iloc[-1]['ATR']
    support = df['low'].rolling(10).min().iloc[-1]
    resistance = df['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return round(stop_loss, 2), round(take_profit, 2)

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None
    
    if df.iloc[-1]['ATR'] < 4.15:
        if df.iloc[-1]['high'] > df.iloc[-2]['high']:
            signal = "BUY"
            entry_price = df.iloc[-1]['close']
        elif df.iloc[-1]['low'] < df.iloc[-2]['low']:
            signal = "SELL"
            entry_price = df.iloc[-1]['close']

        if signal:
            stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)
            log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def equity_check():
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return
    
    inital_balance = account_info.balance
    target_equity = inital_balance + 100
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is not None:
            equity = account_info.equity

            if equity >= target_equity:
                print("🚀 Equity target reached! Closing all open positions...")
                positions = mt5.positions_get()
                if positions:
                    for pos in positions:
                        symbol = pos.symbol
                        volume = pos.volume
                        ticket = pos.ticket
                        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                        price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                        close_request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": ticket,
                            "symbol": symbol,
                            "volume": volume,
                            "type": order_type,
                            "price": price,
                            "deviation": 10,
                            "magic": 123456,
                            "comment": "Auto close on equity profit"
                        }

                        result = mt5.order_send(close_request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"✔️ Closed position {ticket}")
                        else:
                            print(f"❌ Failed to close position {ticket}, retcode: {result.retcode}")
        else:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")

        time.sleep(60)

# === RUN THE BOT ===
def run_bot():
    connect_mt5() # Target profit in USD
    INITIAL_BALANCE = mt5.account_info().balance  # Get initial equity of the account

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check, daemon=True).start()
    
    while True:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Check if equity has reached the target profit
        current_equity = mt5.account_info().equity
        if current_equity >= INITIAL_BALANCE + PROFIT_TARGET:
            print(f"✅ Equity reached ${PROFIT_TARGET} profit. Closing all positions...")
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": pos.ticket,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "price": mt5.symbol_info_tick(SYMBOL).ask if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).bid,
                    "deviation": 10,
                    "magic": 123456,
                    "comment": "Closing positions after reaching profit target",
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    print(f"❌ Failed to close position {pos.ticket}: {result.retcode}")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "SELL":
                        # Close BUY position if SELL signal received
                        close_order = mt5.order_send({
                            "action": mt5.TRADE_ACTION_DEAL,
                            "symbol": pos.symbol,
                            "volume": pos.volume,
                            "type": mt5.ORDER_TYPE_SELL,
                            "position": pos.ticket,
                            "price": mt5.symbol_info_tick(pos.symbol).bid,
                            "deviation": 10,
                            "magic": 1001,
                            "comment": "Auto-close BUY on SELL signal",
                        })
                        if close_order.retcode != mt5.TRADE_RETCODE_DONE:
                            print(f"❌ Failed to close BUY position #{pos.ticket}: {close_order.retcode}")
                        else:
                            print(f"✅ Closed BUY position #{pos.ticket} on SELL signal")

                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "BUY":
                        # Close SELL position if BUY signal received
                        close_order = mt5.order_send({
                            "action": mt5.TRADE_ACTION_DEAL,
                            "symbol": pos.symbol,
                            "volume": pos.volume,
                            "type": mt5.ORDER_TYPE_BUY,
                            "position": pos.ticket,
                            "price": mt5.symbol_info_tick(pos.symbol).ask,
                            "deviation": 10,
                            "magic": 1001,
                            "comment": "Auto-close SELL on BUY signal",
                        })
                        if close_order.retcode != mt5.TRADE_RETCODE_DONE:
                            print(f"❌ Failed to close SELL position #{pos.ticket}: {close_order.retcode}")
                        else:
                            print(f"✅ Closed SELL position #{pos.ticket} on BUY signal")


            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")

        time.sleep(300)


# Start the trading bot
run_bot()


### High volatility, specific SL range

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MAGIC_NUMBER = 10003
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 200  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === CHECK PROFIT AND CLOSE ALL TRADES ===
def check_profit_and_exit():
    global INITIAL_BALANCE
    account_info = mt5.account_info()
    if account_info is None:
        logging.error("❌ Could not fetch account info.")
        return

    net_profit = account_info.balance - INITIAL_BALANCE
    logging.info(f"💹 Current Net Profit: ${net_profit:.2f}")

    if net_profit >= PROFIT_TARGET:
        logging.info("🎯 Profit target reached. Closing all positions...")
        positions = mt5.positions_get()
        if positions:
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": pos.symbol,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "position": pos.ticket,
                    "price": mt5.symbol_info_tick(pos.symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(pos.symbol).ask,
                    "deviation": 20,
                    "magic": MAGIC_NUMBER,
                    "comment": "Close on profit target",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    logging.info(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    logging.warning(f"⚠️ Failed to close position {pos.ticket}: {result.retcode}")
        logging.info("🛑 Stopping bot after reaching profit target.")
        mt5.shutdown()
        exit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(df, period=14):
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = df['true_range'].rolling(period).mean()
    return df

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    sl_pips = 700
    sl_distance = sl_pips * LOT_SIZE

    if signal == "BUY":
        stop_loss = entry_price - sl_distance
        take_profit = entry_price + (sl_distance * 2)
    else:
        stop_loss = entry_price + sl_distance
        take_profit = entry_price - (sl_distance * 2)

    return round(stop_loss, 2), round(take_profit, 2)


# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None
    
    if df.iloc[-1]['ATR'] > 4.5:
        if df.iloc[-1]['high'] > df.iloc[-2]['high']:
            signal = "BUY"
            entry_price = df.iloc[-1]['close']
        elif df.iloc[-1]['low'] < df.iloc[-2]['low']:
            signal = "SELL"
            entry_price = df.iloc[-1]['close']

        if signal:
            stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)
            log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def equity_check():
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return
    
    inital_balance = account_info.balance
    target_equity = inital_balance + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is not None:
            equity = account_info.equity

            if equity >= target_equity:
                print("🚀 Equity target reached! Closing all open positions...")
                positions = mt5.positions_get()
                if positions:
                    for pos in positions:
                        symbol = pos.symbol
                        volume = pos.volume
                        ticket = pos.ticket
                        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                        price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                        close_request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": ticket,
                            "symbol": symbol,
                            "volume": volume,
                            "type": order_type,
                            "price": price,
                            "deviation": 10,
                            "magic": 123456,
                            "comment": "Auto close on equity profit"
                        }

                        result = mt5.order_send(close_request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"✔️ Closed position {ticket}")
                        else:
                            print(f"❌ Failed to close position {ticket}, retcode: {result.retcode}")
        else:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")

        time.sleep(60)

# === RUN THE BOT ===
def run_bot():
    connect_mt5() # Target profit in USD
    INITIAL_BALANCE = mt5.account_info().balance  # Get initial equity of the account

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check, daemon=True).start()
    
    while True:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Check if equity has reached the target profit
        current_equity = mt5.account_info().equity
        if current_equity >= INITIAL_BALANCE + PROFIT_TARGET:
            print(f"✅ Equity reached ${PROFIT_TARGET} profit. Closing all positions...")
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": pos.ticket,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "price": mt5.symbol_info_tick(SYMBOL).ask if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(SYMBOL).bid,
                    "deviation": 10,
                    "magic": 123456,
                    "comment": "Closing positions after reaching profit target",
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    print(f"❌ Failed to close position {pos.ticket}: {result.retcode}")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "SELL":
                        # Close BUY position if SELL signal received
                        close_order = mt5.order_send({
                            "action": mt5.TRADE_ACTION_DEAL,
                            "symbol": pos.symbol,
                            "volume": pos.volume,
                            "type": mt5.ORDER_TYPE_SELL,
                            "position": pos.ticket,
                            "price": mt5.symbol_info_tick(pos.symbol).bid,
                            "deviation": 10,
                            "magic": 1001,
                            "comment": "Auto-close BUY on SELL signal",
                        })
                        if close_order.retcode != mt5.TRADE_RETCODE_DONE:
                            print(f"❌ Failed to close BUY position #{pos.ticket}: {close_order.retcode}")
                        else:
                            print(f"✅ Closed BUY position #{pos.ticket} on SELL signal")

                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "BUY":
                        # Close SELL position if BUY signal received
                        close_order = mt5.order_send({
                            "action": mt5.TRADE_ACTION_DEAL,
                            "symbol": pos.symbol,
                            "volume": pos.volume,
                            "type": mt5.ORDER_TYPE_BUY,
                            "position": pos.ticket,
                            "price": mt5.symbol_info_tick(pos.symbol).ask,
                            "deviation": 10,
                            "magic": 1001,
                            "comment": "Auto-close SELL on BUY signal",
                        })
                        if close_order.retcode != mt5.TRADE_RETCODE_DONE:
                            print(f"❌ Failed to close SELL position #{pos.ticket}: {close_order.retcode}")
                        else:
                            print(f"✅ Closed SELL position #{pos.ticket} on BUY signal")


            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")

        time.sleep(300)


# Start the trading bot
run_bot()


✅ MT5 initialized and logged in successfully.
✅ Initial Balance: 1152.0
🎯 Monitoring for equity to reach $1352.00...
📢 Trade Signal: BUY | Entry: 3294.65 | SL: 3287.65 | TP: 3308.65
✅ BUY Order Placed: 3294.65
✅ SL/TP Updated: SL=3287.65, TP=3308.65
📢 Trade Signal: BUY | Entry: 3294.262 | SL: 3287.26 | TP: 3308.26
✅ BUY Order Placed: 3294.262
✅ SL/TP Updated: SL=3287.26, TP=3308.26
⏳ No trade opportunity found.
⏳ No trade opportunity found.
✅ Closed BUY position #1881058381 on SELL signal
✅ Closed BUY position #1881076611 on SELL signal
📢 Trade Signal: SELL | Entry: 3292.157 | SL: 3299.16 | TP: 3264.16
✅ SELL Order Placed: 3292.157
✅ SL/TP Updated: SL=3299.16, TP=3264.16
✅ SELL SL updated for position 1881134387 → 3294.17
📢 Trade Signal: SELL | Entry: 3287.168 | SL: 3294.17 | TP: 3259.17
✅ SELL Order Placed: 3287.168
✅ SL/TP Updated: SL=3294.17, TP=3259.17
✅ SELL SL updated for position 1881134387 → 3289.15
✅ SELL SL updated for position 1881161038 → 3294.14
📢 Trade Signal: SELL | Entr

### Retcode modification (working)

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MAGIC_NUMBER = 10005
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 200  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === CHECK PROFIT AND CLOSE ALL TRADES ===
def check_profit_and_exit():
    global INITIAL_BALANCE
    account_info = mt5.account_info()
    if account_info is None:
        logging.error("❌ Could not fetch account info.")
        return

    net_profit = account_info.balance - INITIAL_BALANCE
    logging.info(f"💹 Current Net Profit: ${net_profit:.2f}")

    if net_profit >= PROFIT_TARGET:
        logging.info("🎯 Profit target reached. Closing all positions...")
        positions = mt5.positions_get()
        if positions:
            for pos in positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": pos.symbol,
                    "volume": pos.volume,
                    "type": mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "position": pos.ticket,
                    "price": mt5.symbol_info_tick(pos.symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(pos.symbol).ask,
                    "deviation": 20,
                    "magic": MAGIC_NUMBER,
                    "comment": "Close on profit target",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC
                }
                result = mt5.order_send(close_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    logging.info(f"✅ Position {pos.ticket} closed successfully.")
                else:
                    logging.warning(f"⚠️ Failed to close position {pos.ticket}: {result.retcode}")
        logging.info("🛑 Stopping bot after reaching profit target.")
        mt5.shutdown()
        exit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(df, period=14):
    df['high-low'] = df['high'] - df['low']
    df['high-close'] = abs(df['high'] - df['close'].shift(1))
    df['low-close'] = abs(df['low'] - df['close'].shift(1))
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    df['ATR'] = df['true_range'].rolling(period).mean()
    return df

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(df, entry_price, signal):
    atr = df.iloc[-1]['ATR']
    support = df['low'].rolling(10).min().iloc[-1]
    resistance = df['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return round(stop_loss, 2), round(take_profit, 2)

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(df):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if df.iloc[-1]['high'] > df.iloc[-2]['high']:
        signal = "BUY"
        entry_price = df.iloc[-1]['close']
    elif df.iloc[-1]['low'] < df.iloc[-2]['low']:
        signal = "SELL"
        entry_price = df.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(df, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# == RETCODES ==
RETCODE_MESSAGES = {
    10004: "Requote",
    10006: "Request rejected",
    10007: "Request canceled by trader",
    10011: "Request processing error",
    10012: "Request canceled by timeout",
    10013: "Invalid request",
    10014: "Invalid volume in the request",
    10015: "Invalid price in the request",
    10016: "Invalid stops in the request",
    10017: "Trade is disabled",
    10018: "Market is closed",
    10019: "Not enough money to complete the request",
    10020: "Prices changed",
    10021: "No quotes to process the request",
    10022: "Invalid order expiration date",
    10023: "Order state changed",
    10024: "Too frequent requests",
    10025: "No changes in request",
    10026: "Autotrading disabled by server",
    10027: "Autotrading disabled by client terminal",
    10028: "Request locked for processing",
    10029: "Order or position frozen",
    10030: "Invalid order filling type",
    10031: "No connection with trade server",
    10032: "Only allowed for live accounts",
    10033: "Pending orders limit reached",
    10034: "Order/position volume limit reached for symbol",
    10035: "Incorrect or prohibited order type",
    10036: "Position already closed",
    10038: "Close volume exceeds position volume",
    10039: "Close order already exists for position",
    10040: "Open positions limit reached",
    10041: "Activation rejected, order canceled",
    10042: "Only long positions allowed",
    10043: "Only short positions allowed",
    10044: "Only position closing allowed",
    10045: "Only FIFO closing allowed",
    10046: "Opposite positions disabled (hedging prohibited)",
}

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:  # Ensure the order is not None
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                modify_message = RETCODE_MESSAGES.get(modify.retcode, str(modify.retcode))
                print(f"❌ Failed to update SL/TP: {modify_message}")
        else:
            order_message = RETCODE_MESSAGES.get(order.retcode, str(order.retcode))
            print(f"❌ Trade Failed: {order_message}")
    else:
        print("❌ Order send failed: order is None")
     
def equity_check():
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return
    
    inital_balance = account_info.balance
    target_equity = inital_balance + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is not None:
            equity = account_info.equity

            if equity >= target_equity:
                print("🚀 Equity target reached! Closing all open positions...")
                positions = mt5.positions_get()
                if positions:
                    for pos in positions:
                        symbol = pos.symbol
                        volume = pos.volume
                        ticket = pos.ticket
                        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                        price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

                        close_request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": ticket,
                            "symbol": symbol,
                            "volume": volume,
                            "type": order_type,
                            "price": price,
                            "deviation": 10,
                            "magic": 123456,
                            "comment": "Auto close on equity profit"
                        }

                        result = mt5.order_send(close_request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"✔️ Closed position {ticket}")
                        else:
                            error_message = RETCODE_MESSAGES.get(result.retcode, str(result.retcode))
                            print(f"❌ Failed to close position {ticket}, retcode: {error_message}")
        else:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")

        time.sleep(60)

# === RUN THE BOT ===
def run_bot():
    connect_mt5()
    target_profit = 200  # Target profit in USD
    initial_equity = mt5.account_info().balance  # Get initial equity of the account

    # Start the equity check in a separate thread
    threading.Thread(target=equity_check, daemon=True).start()
    
    while True:
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions
        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    # == Trailing SL for Buy position ==
                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            error_message = RETCODE_MESSAGES.get(result.retcode, str(result.retcode)) # Converts the error codes to describtions
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {error_message}")

                    # == Trailing SL for Sell position ==
                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            error_message = RETCODE_MESSAGES.get(result.retcode, str(result.retcode))
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {error_message}")
            else:
                print("❌ Failed to fetch current price tick.")

        # ✅ Trade only if signal exists and allowed
        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    # == Skip trade if there is already an open position of same type ==
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")

        time.sleep(300)


# Start the trading bot
run_bot()


### Original (Don't run)

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
LOT_SIZE = 0.01
MAGIC_NUMBER = 10002
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 200  # 💰 Target profit in USD
INITIAL_BALANCE = None  # 🔒 Will be set after login


# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242541106,
        password="Password123.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()

    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')


# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    dataframe = pd.DataFrame(rates)
    dataframe['time'] = pd.to_datetime(dataframe['time'], unit='s')
    return dataframe


# === CALCULATE ATR (Average True Range) ===
def calculate_atr(dataframe, period=14):
    dataframe['high-low'] = dataframe['high'] - dataframe['low']
    dataframe['high-close'] = abs(dataframe['high'] - dataframe['close'].shift(1))
    dataframe['low-close'] = abs(dataframe['low'] - dataframe['close'].shift(1))
    dataframe['true_range'] = dataframe[['high-low', 'high-close', 'low-close']].max(axis=1)
    dataframe['ATR'] = dataframe['true_range'].rolling(period).mean()
    return dataframe


# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(dataframe, entry_price, signal):
    atr = dataframe.iloc[-1]['ATR']
    support = dataframe['low'].rolling(10).min().iloc[-1]
    resistance = dataframe['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return round(stop_loss, 2), round(take_profit, 2)


# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")


# === IDENTIFY TRADE SIGNALS ===
def generate_signals(dataframe):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if dataframe.iloc[-1]['high'] > dataframe.iloc[-2]['high']:
        signal = "BUY"
        entry_price = dataframe.iloc[-1]['close']
    elif dataframe.iloc[-1]['low'] < dataframe.iloc[-2]['low']:
        signal = "SELL"
        entry_price = dataframe.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(dataframe, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit


# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: {entry_price}")
            position_id = order.order
            time.sleep(2)
            modify_request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "position": position_id,
                "sl": stop_loss,
                "tp": take_profit
            }
            modify = mt5.order_send(modify_request)
            if modify.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"✅ SL/TP Updated: SL={stop_loss}, TP={take_profit}")
            else:
                print(f"❌ Failed to update SL/TP: {modify.retcode}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")


# === MONITOR EQUITY TARGET ===
def equity_check():
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return

    inital_balance = account_info.balance
    target_equity = INITIAL_BALANCE + PROFIT_TARGET
    print(f"🎯 Monitoring for equity to reach ${target_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is not None:
            equity = account_info.equity
            if equity >= target_equity:
                print("🚀 Equity target reached! Closing all open positions...")
                positions = mt5.positions_get()
                if positions:
                    for pos in positions:
                        symbol = pos.symbol
                        volume = pos.volume
                        ticket = pos.ticket
                        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                        price = (
                            mt5.symbol_info_tick(symbol).bid
                            if pos.type == mt5.ORDER_TYPE_BUY
                            else mt5.symbol_info_tick(symbol).ask
                        )

                        close_request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": ticket,
                            "symbol": symbol,
                            "volume": volume,
                            "type": order_type,
                            "price": price,
                            "deviation": 10,
                            "magic": 123456,
                            "comment": "Auto close on equity profit"
                        }

                        result = mt5.order_send(close_request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"✔️ Closed position {ticket}")
                        else:
                            print(f"❌ Failed to close position {ticket}, retcode: {result.retcode}")
        else:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")

        time.sleep(60)


# === RUN THE BOT ===
def run_bot():
    connect_mt5()

    threading.Thread(target=equity_check, daemon=True).start()

    while True:
        dataframe = get_data(SYMBOL, TIMEFRAME, 100)
        if dataframe is None or dataframe.empty:
            print("❌ No data received, retrying...")
            time.sleep(300)
            continue

        dataframe = calculate_atr(dataframe)
        signal, entry_price, stop_loss, take_profit = generate_signals(dataframe)

        positions = mt5.positions_get(symbol=SYMBOL)
        if positions:
            tick = mt5.symbol_info_tick(SYMBOL)
            if tick:
                for pos in positions:
                    entry = pos.price_open
                    current_sl = pos.sl
                    sl_distance = abs(entry - current_sl)

                    if pos.type == mt5.ORDER_TYPE_BUY:
                        current_price = tick.ask
                        new_sl = round(current_price - sl_distance, 2)
                        if current_price > entry and new_sl > current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update BUY SL: {result.retcode}")

                    elif pos.type == mt5.ORDER_TYPE_SELL:
                        current_price = tick.bid
                        new_sl = round(current_price + sl_distance, 2)
                        if current_price < entry and new_sl < current_sl:
                            modify_request = {
                                "action": mt5.TRADE_ACTION_SLTP,
                                "position": pos.ticket,
                                "sl": new_sl,
                                "tp": pos.tp
                            }
                            result = mt5.order_send(modify_request)
                            if result.retcode == mt5.TRADE_RETCODE_DONE:
                                print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                            else:
                                print(f"❌ Failed to update SELL SL: {result.retcode}")
            else:
                print("❌ Failed to fetch current price tick.")

        if signal:
            allow_trade = True
            if positions:
                for pos in positions:
                    if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                        print("⚠️ Trade skipped: Already in a BUY position.")
                        allow_trade = False
                        break
                    elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                        print("⚠️ Trade skipped: Already in a SELL position.")
                        allow_trade = False
                        break

            if allow_trade:
                print(f"📢 Trade Signal: {signal} | Entry: {entry_price}")
                place_trade(signal, entry_price, stop_loss, take_profit)
            else:
                print("⏳ Waiting for new opposite signal...")
        else:
            print("⏳ No trade opportunity found.")

        time.sleep(300)


# Start the trading bot
run_bot()

### 1st Modification for single accounts

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
timeframe_to_next_candle_minutes = 5
LOT_SIZE = 0.03
MAGIC_NUMBER = 10002
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 600  # 💰 Target profit in USD
LOSS_TARGET = 600
INITIAL_BALANCE = None  # 🔒 Will be set after login
stop_trading = False  # Shared flag

# Define trading start and end times
start_time = datetime.strptime("10:10", "%H:%M").time()  
end_time = datetime.strptime("6:00", "%H:%M").time()    

# === RETCODE MESSAGES ===
RETCODE_MESSAGES = {
    10004: "Requote",
    10006: "Request rejected",
    10007: "Request canceled by trader",
    10011: "Request processing error",
    10012: "Request canceled by timeout",
    10013: "Invalid request",
    10014: "Invalid volume in the request",
    10015: "Invalid price in the request",
    10016: "Invalid stops in the request",
    10017: "Trade is disabled",
    10018: "Market is closed",
    10019: "Not enough money to complete the request",
    10020: "Prices changed",
    10021: "No quotes to process the request",
    10022: "Invalid order expiration date",
    10023: "Order state changed",
    10024: "Too frequent requests",
    10025: "No changes in request",
    10026: "Autotrading disabled by server",
    10027: "Autotrading disabled by client terminal",
    10028: "Request locked for processing",
    10029: "Order or position frozen",
    10030: "Invalid order filling type",
    10031: "No connection with trade server",
    10032: "Only allowed for live accounts",
    10033: "Pending orders limit reached",
    10034: "Order/position volume limit reached for symbol",
    10035: "Incorrect or prohibited order type",
    10036: "Position already closed",
    10038: "Close volume exceeds position volume",
    10039: "Close order already exists for position",
    10040: "Open positions limit reached",
    10041: "Activation rejected, order canceled",
    10042: "Only long positions allowed",
    10043: "Only short positions allowed",
    10044: "Only position closing allowed",
    10045: "Only FIFO closing allowed",
    10046: "Opposite positions disabled (hedging prohibited)",
}

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === WAIT FOR NEXT CANDLE ===
def wait_for_next_candle():
    now = datetime.now()
    seconds_in_timeframe = timeframe_to_next_candle_minutes * 60
    seconds_since_epoch = int(now.timestamp())
    seconds_to_next_candle = seconds_in_timeframe - (seconds_since_epoch % seconds_in_timeframe)
    time.sleep(seconds_to_next_candle)

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(dataframe):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if dataframe.iloc[-1]['high'] > dataframe.iloc[-2]['high']:
        signal = "BUY"
        entry_price = dataframe.iloc[-1]['close']
    elif dataframe.iloc[-1]['low'] < dataframe.iloc[-2]['low']:
        signal = "SELL"
        entry_price = dataframe.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(dataframe, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(dataframe, period=14):
    dataframe['high-low'] = dataframe['high'] - dataframe['low']
    dataframe['high-close'] = abs(dataframe['high'] - dataframe['close'].shift(1))
    dataframe['low-close'] = abs(dataframe['low'] - dataframe['close'].shift(1))
    dataframe['true_range'] = dataframe[['high-low', 'high-close', 'low-close']].max(axis=1)
    dataframe['ATR'] = dataframe['true_range'].rolling(period).mean()
    return dataframe

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(dataframe, entry_price, signal):
    atr = dataframe.iloc[-1]['ATR']
    support = dataframe['low'].rolling(10).min().iloc[-1]
    resistance = dataframe['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return stop_loss, take_profit

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "sl": stop_loss,
        "tp": take_profit,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: Entry={entry_price}, SL={stop_loss}, TP={take_profit}")
        else:
            error_message = RETCODE_MESSAGES.get(order.retcode, f"Unknown error (retcode: {order.retcode})")
            print(f"❌ Trade Failed: {error_message}")
    else:
        print("❌ Order send failed: order is None")

# === TRADING HOURS CHECK ===
def is_trading_hours():
    now = datetime.now().time()
    if start_time < end_time:
        return start_time <= now <= end_time
    else:  # Overnight session
        return now >= start_time or now <= end_time

# === SLEEP UNTIL TRADING HOURS ===
def sleep_until_trading_hours():
    now = datetime.now()
    today_start = now.replace(hour=start_time.hour, minute=start_time.minute, second=0, microsecond=0)

    if now >= today_start:
        sleep_until = today_start + timedelta(days=1)
    else:
        sleep_until = today_start

    sleep_seconds = (sleep_until - now).total_seconds()
    hours, remainder = divmod(int(sleep_seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"⏸️ Sleeping for {hours}h {minutes}m {seconds}s until {start_time.strftime('%H:%M')}...")
    time.sleep(sleep_seconds)

# === CLOSE ALL POSITIONS ===
def close_all_positions():
    positions = mt5.positions_get()
    if positions is None or len(positions) == 0:
        print("No open positions to close.")
        return

    for pos in positions:
        symbol = pos.symbol
        volume = pos.volume
        ticket = pos.ticket
        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

        close_request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": ticket,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": price,
            "deviation": 10,
            "magic": MAGIC_NUMBER,
            "comment": "Closed at end time"
        }

        result = mt5.order_send(close_request)
        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ Position {ticket} closed successfully.")
        else:
            error_message = RETCODE_MESSAGES.get(result.retcode, f"Unknown error (retcode: {result.retcode})")
            print(f"❌ Failed to close position {ticket}: {error_message}")

# === EQUITY CHECK AND CLOSE TRADES ===
def equity_check_and_close_trades():
    global INITIAL_BALANCE, PROFIT_TARGET, LOSS_TARGET, stop_trading
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return

    target_equity = INITIAL_BALANCE + PROFIT_TARGET
    loss_equity = INITIAL_BALANCE - LOSS_TARGET
    print(f"🎯 Monitoring equity: Target ${target_equity:.2f}, Max-loss ${loss_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is None:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")
            wait_for_next_candle()
            continue

        equity = account_info.equity

        if equity >= target_equity:
            print("🚀 Equity profit target reached! Closing all open positions...")
            close_all_positions()
            stop_trading = True
            break

        elif equity <= loss_equity:
            print("🛑 Equity loss limit reached! Closing all open positions...")
            close_all_positions()
            stop_trading = True
            break

        time.sleep(10)

# === TRAILING STOP LOSS ===
def trailing_stop_loss(symbol):
    positions = mt5.positions_get(symbol=symbol)
    if not positions:
        return

    tick = mt5.symbol_info_tick(symbol)
    if not tick:
        print("❌ Failed to fetch current price tick.")
        return

    for pos in positions:
        entry = pos.price_open
        current_sl = pos.sl
        sl_distance = abs(entry - current_sl)

        if pos.type == mt5.ORDER_TYPE_BUY:
            current_price = tick.ask
            new_sl = round(current_price - sl_distance, 2)
            if current_price > entry and new_sl > current_sl:
                modify_request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": pos.ticket,
                    "sl": new_sl,
                    "tp": pos.tp
                }
                result = mt5.order_send(modify_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                else:
                    error_message = RETCODE_MESSAGES.get(result.retcode, f"Unknown error (retcode: {result.retcode})")
                    print(f"❌ Failed to update BUY SL: {error_message}")

        elif pos.type == mt5.ORDER_TYPE_SELL:
            current_price = tick.bid
            new_sl = round(current_price + sl_distance, 2)
            if current_price < entry and new_sl < current_sl:
                modify_request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": pos.ticket,
                    "sl": new_sl,
                    "tp": pos.tp
                }
                result = mt5.order_send(modify_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                else:
                    error_message = RETCODE_MESSAGES.get(result.retcode, f"Unknown error (retcode: {result.retcode})")
                    print(f"❌ Failed to update SELL SL: {error_message}")

# === TRADE ON SIGNAL ===
def trade_on_signal(signal, entry_price, stop_loss, take_profit, symbol):
    if signal:
        positions = mt5.positions_get(symbol=symbol)
        allow_trade = True

        if positions:
            for pos in positions:
                if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                    print("⚠️ Trade skipped: Already in a BUY position.")
                    allow_trade = False
                    break
                elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                    print("⚠️ Trade skipped: Already in a SELL position.")
                    allow_trade = False
                    break

        if allow_trade:
            print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
            place_trade(signal, entry_price, stop_loss, take_profit)
        else:
            print("⏳ Waiting for new opposite signal...")
    else:
        print("⏳ No trade opportunity found.")

# === RUN THE BOT ===
def run_bot():
    global stop_trading
    connect_mt5()
    
    if not is_trading_hours():
        print("🕒 Trading paused — market is currently outside the active hours window")
        sleep_until_trading_hours()

    threading.Thread(target=equity_check_and_close_trades, daemon=True).start()

    while True:
        if stop_trading:
            sleep_until_trading_hours()
            break
        
        if not is_trading_hours():
            print("🛑 Trading hours ended. Closing all positions...")
            close_all_positions()
            sleep_until_trading_hours()
            continue
        
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            wait_for_next_candle()
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        trailing_stop_loss(SYMBOL)
                
        trade_on_signal(signal, entry_price, stop_loss, take_profit, SYMBOL)
        
        wait_for_next_candle()

# Start the trading bot
run_bot()

### 2ND Modification for Multi accounts

In [ ]:
# === LOGGER SETUP ===
import logging

# Set up logging with UTF-8 encoding
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()])
logging.getLogger().handlers[0].setStream(open('CON', 'w', encoding='utf-8'))

logging.info("✅ MT5 initialized successfully.")

import MetaTrader5 as mt5
import pandas as pd
import time
import talib
import threading
from datetime import datetime, timedelta

# === CONSTANTS ===
SYMBOL = "XAUUSDm"
TIMEFRAME = mt5.TIMEFRAME_M15
timeframe_to_next_candle_minutes = 5
LOT_SIZE = 0.03
MAGIC_NUMBER = 10002
LOG_FILE = "trade_log.txt"
PROFIT_TARGET = 30  # 💰 Target profit in USD
LOSS_TARGET = 30
INITIAL_BALANCE = None  # 🔒 Will be set after login
stop_trading = False  # Shared flag

# Define trading start and end times
start_time = datetime.strptime("10:10", "%H:%M").time()  
end_time = datetime.strptime("6:00", "%H:%M").time()    

# === MT5 CONNECT ===
def connect_mt5():
    global INITIAL_BALANCE
    if not mt5.initialize(
        login=242864620,
        password="Password1234.",
        server="Exness-MT5Trial"
    ):
        print("❌ MT5 initialization/login failed!")
        quit()
        
    print("✅ MT5 initialized and logged in successfully.")
    account_info = mt5.account_info()
    INITIAL_BALANCE = account_info.balance
    print(f'✅ Initial Balance: {INITIAL_BALANCE}')
    if not account_info:
        print("❌ Unable to fetch account info.")
        quit()

# === (Rest of your existing code remains unchanged — insert check below into main loop) ===

def wait_for_next_candle():
    now = datetime.now()
    seconds_in_timeframe = timeframe_to_next_candle_minutes * 60
    seconds_since_epoch = int(now.timestamp())
    seconds_to_next_candle = seconds_in_timeframe - (seconds_since_epoch % seconds_in_timeframe)
    # print(f"⏳ Waiting {seconds_to_next_candle}s for next candle...")
    time.sleep(seconds_to_next_candle)

# === IDENTIFY TRADE SIGNALS ===
def generate_signals(dataframe):
    signal = None
    entry_price = None
    stop_loss = None
    take_profit = None

    if dataframe.iloc[-1]['high'] > dataframe.iloc[-2]['high']:
        signal = "BUY"
        entry_price = dataframe.iloc[-1]['close']
    elif dataframe.iloc[-1]['low'] < dataframe.iloc[-2]['low']:
        signal = "SELL"
        entry_price = dataframe.iloc[-1]['close']

    if signal:
        stop_loss, take_profit = predict_sl_tp(dataframe, entry_price, signal)
        log_trade_signal(signal, entry_price, stop_loss, take_profit)

    return signal, entry_price, stop_loss, take_profit

# === FETCH MARKET DATA ===
def get_data(symbol, timeframe, count=100):
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
    if rates is None:
        print(f"❌ No data for {symbol}. Check symbol name!")
        return None
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

# === CALCULATE ATR (Average True Range) ===
def calculate_atr(dataframe, period=14):
    dataframe['high-low'] = dataframe['high'] - dataframe['low']
    dataframe['high-close'] = abs(dataframe['high'] - dataframe['close'].shift(1))
    dataframe['low-close'] = abs(dataframe['low'] - dataframe['close'].shift(1))
    dataframe['true_range'] = dataframe[['high-low', 'high-close', 'low-close']].max(axis=1)
    dataframe['ATR'] = dataframe['true_range'].rolling(period).mean()
    return dataframe

# === AI-BASED SL/TP CALCULATION ===
def predict_sl_tp(dataframe, entry_price, signal):
    atr = dataframe.iloc[-1]['ATR']
    support = dataframe['low'].rolling(10).min().iloc[-1]
    resistance = dataframe['high'].rolling(10).max().iloc[-1]

    if signal == "BUY":
        stop_loss = max(support, entry_price - (atr * 1.5))
        take_profit = entry_price + ((entry_price - stop_loss) * 2)
    else:
        stop_loss = min(resistance, entry_price + (atr * 1.5))
        take_profit = entry_price - ((stop_loss - entry_price) * 2)

    return stop_loss, take_profit

# === LOG TRADE SIGNAL ===
def log_trade_signal(signal, entry_price, stop_loss, take_profit):
    with open(LOG_FILE, "a") as file:
        file.write(f"{datetime.now()} | {SYMBOL} | {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}\n")

# === PLACE TRADE ===
def place_trade(signal, entry_price, stop_loss, take_profit):
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": SYMBOL,
        "volume": LOT_SIZE,
        "type": mt5.ORDER_TYPE_BUY if signal == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": entry_price,
        "sl": stop_loss,
        "tp": take_profit,
        "deviation": 20,
        "magic": MAGIC_NUMBER,
        "comment": f"SMC {signal}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    order = mt5.order_send(request)

    if order is not None:
        if order.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ {signal} Order Placed: Entry={entry_price}, SL={stop_loss}, TP={take_profit}")
        else:
            print(f"❌ Trade Failed: {order.retcode}")
    else:
        print("❌ Order send failed: order is None")
        
def is_trading_hours():
    now = datetime.now().time()
    if start_time < end_time:
        return start_time <= now <= end_time
    else:  # Overnight session
        return now >= start_time or now <= end_time

def sleep_until_trading_hours():
    now = datetime.now()
    today_start = now.replace(hour=start_time.hour, minute=start_time.minute, second=0, microsecond=0)

    # If it's already past today's start_time, schedule for next day's start_time
    if now >= today_start:
        sleep_until = today_start + timedelta(days=1)
    else:
        sleep_until = today_start

    sleep_seconds = (sleep_until - now).total_seconds()
    hours, remainder = divmod(int(sleep_seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"⏸️ Sleeping for {hours}h {minutes}m {seconds}s until {start_time.strftime('%H:%M')}...")
    time.sleep(sleep_seconds)

def close_all_positions():
    positions = mt5.positions_get()
    if positions is None or len(positions) == 0:
        print("No open positions to close.")
        return

    for pos in positions:
        symbol = pos.symbol
        volume = pos.volume
        ticket = pos.ticket
        order_type = mt5.ORDER_TYPE_SELL if pos.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).bid if pos.type == mt5.ORDER_TYPE_BUY else mt5.symbol_info_tick(symbol).ask

        close_request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": ticket,
            "symbol": symbol,
            "volume": volume,
            "type": order_type,
            "price": price,
            "deviation": 10,
            "magic": MAGIC_NUMBER,
            "comment": "Closed at end time"
        }

        result = mt5.order_send(close_request)
        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"✅ Position {ticket} closed successfully.")
        else:
            print(f"❌ Failed to close position {ticket}: {result.retcode}")

def equity_check_and_close_trades():
    global INITIAL_BALANCE, PROFIT_TARGET, LOSS_TARGET, stop_trading
    account_info = mt5.account_info()
    if account_info is None:
        print(f"❌ Unable to retrieve account info. Error: {mt5.last_error()}")
        return

    target_equity = INITIAL_BALANCE + PROFIT_TARGET
    loss_equity = INITIAL_BALANCE - LOSS_TARGET
    print(f"🎯 Monitoring equity: Target ${target_equity:.2f}, Max-loss ${loss_equity:.2f}...")

    while True:
        account_info = mt5.account_info()
        if account_info is None:
            print(f"⚠️ account_info() returned None — error: {mt5.last_error()}")
            wait_for_next_candle()
            continue

        equity = account_info.equity

        if equity >= target_equity:
            print("🚀 Equity profit target reached! Closing all open positions...")
            close_all_positions()
            stop_trading = True
            # sleep_until_trading_hours()
              # 🔴 Set flag to stop the main loop
            break

        elif equity <= loss_equity:
            print("🛑 Equity loss limit reached! Closing all open positions...")
            close_all_positions()
            stop_trading = True
            # sleep_until_trading_hours()
              # 🔴 Set flag to stop the main loop
            break

        time.sleep(10)

def trailing_stop_loss(symbol):
    positions = mt5.positions_get(symbol=symbol)
    if not positions:
        return

    tick = mt5.symbol_info_tick(symbol)
    if not tick:
        print("❌ Failed to fetch current price tick.")
        return

    for pos in positions:
        entry = pos.price_open
        current_sl = pos.sl
        sl_distance = abs(entry - current_sl)

        if pos.type == mt5.ORDER_TYPE_BUY:
            current_price = tick.ask
            new_sl = current_price - sl_distance
            if current_price > entry and new_sl > current_sl:
                modify_request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": pos.ticket,
                    "sl": new_sl,
                    "tp": pos.tp
                }
                result = mt5.order_send(modify_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ BUY SL updated for position {pos.ticket} → {new_sl}")
                else:
                    print(f"❌ Failed to update BUY SL: {result.retcode}")

        elif pos.type == mt5.ORDER_TYPE_SELL:
            current_price = tick.bid
            new_sl = current_price + sl_distance
            if current_price < entry and new_sl < current_sl:
                modify_request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "position": pos.ticket,
                    "sl": new_sl,
                    "tp": pos.tp
                }
                result = mt5.order_send(modify_request)
                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"✅ SELL SL updated for position {pos.ticket} → {new_sl}")
                else:
                    print(f"❌ Failed to update SELL SL: {result.retcode}")

def trade_on_signal(signal, entry_price, stop_loss, take_profit, symbol):
    if signal:
        positions = mt5.positions_get(symbol=symbol)
        allow_trade = True

        if positions:
            for pos in positions:
                if pos.type == mt5.ORDER_TYPE_BUY and signal == "BUY":
                    print("⚠️ Trade skipped: Already in a BUY position.")
                    allow_trade = False
                    break
                elif pos.type == mt5.ORDER_TYPE_SELL and signal == "SELL":
                    print("⚠️ Trade skipped: Already in a SELL position.")
                    allow_trade = False
                    break

        if allow_trade:
            print(f"📢 Trade Signal: {signal} | Entry: {entry_price} | SL: {stop_loss} | TP: {take_profit}")
            place_trade(signal, entry_price, stop_loss, take_profit)
        else:
            print("⏳ Waiting for new opposite signal...")
    else:
        print("⏳ No trade opportunity found.")


# === RUN THE BOT ===
def run_bot():
    
    global stop_trading
    connect_mt5()
    
    # Pause the bot if it's outside the trading window
    if not is_trading_hours():
        print("🕒 Trading paused — market is currently outside the active hours window")
        sleep_until_trading_hours()

    # Start Threads
    threading.Thread(target=equity_check_and_close_trades, daemon=True).start()  # Equity check to monitor account equity and close trades if needed

    # Main trading loop runs until stop condition is triggered
    while True:
        if stop_trading:
            sleep_until_trading_hours()
            break
        
        # Check for trading hours
        if not is_trading_hours():
            print("🛑 Trading hours ended. Closing all positions...")
            close_all_positions()
            sleep_until_trading_hours()
            continue
        
        # Get market data
        df = get_data(SYMBOL, TIMEFRAME, 100)
        if df is None or df.empty:
            print("❌ No data received, retrying...")
            wait_for_next_candle()
            continue

        df = calculate_atr(df)
        signal, entry_price, stop_loss, take_profit = generate_signals(df)

        # ✅ Always check and update SL for open positions== Trailing stop loss
        trailing_stop_loss(SYMBOL)
                
        # ✅ Trade only if signal exists and allowed
        trade_on_signal(signal, entry_price, stop_loss, take_profit, SYMBOL)
        
        wait_for_next_candle()

# Start the trading bot
run_bot()


## tests

In [35]:
import MetaTrader5 as mt5

def place_trade(symbol, lot, sl_pips):
    if not mt5.initialize():
        print("initialize() failed, error code =", mt5.last_error())
        return

    symbol_info = mt5.symbol_info(symbol)
    tick = mt5.symbol_info_tick(symbol)

    if symbol_info is None or tick is None:
        print("Failed to get symbol info or tick data.")
        mt5.shutdown()
        return

    if not symbol_info.visible:
        if not mt5.symbol_select(symbol, True):
            print(f"symbol_select({symbol}) failed.")
            mt5.shutdown()
            return

    price = tick.ask
    point = 1
    sl_price = price - sl_pips * point

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": sl_price,
        "deviation": 10,
        "magic": 1001,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC
    }

    result = mt5.order_send(request)

    if result is None:
        print("order_send() returned None. Error:", mt5.last_error())
    elif result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Order failed. Retcode: {result.retcode}")
    else:
        print(f"Order placed successfully. Ticket: {result.order}")

    mt5.shutdown()


In [ ]:
place_trade("BTCUSDm", 0.01, 15)

Order placed successfully. Ticket: 1944080210
